In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
SEASON = '21'

In [7]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [8]:
from fastai.tabular.all import * 
from utils import *

In [9]:
path_raw = Path('raw_data')
path_data = Path('data')

In [10]:
!ls -lah -t {str(path_raw)}

total 9880
drwxr-xr-x  25 twtang  staff   800B Nov  7 08:18 21
drwxr-xr-x  21 twtang  staff   672B Nov  7 08:15 ..
-rw-r--r--@  1 twtang  staff    10K Nov  5 21:57 .DS_Store
-rw-r--r--   1 twtang  staff   336K Nov  5 06:49 data.zip
drwxr-xr-x  18 twtang  staff   576B Oct 30 14:35 .
drwx------@ 24 twtang  staff   768B Oct 30 14:35 20
drwx------@ 24 twtang  staff   768B Oct 30 14:35 19
drwx------@ 24 twtang  staff   768B Oct 30 14:35 18
drwx------@ 24 twtang  staff   768B Oct 30 14:35 17
drwx------@ 24 twtang  staff   768B Oct 30 14:35 16
drwx------@ 24 twtang  staff   768B Oct 30 14:34 15
-rw-r--r--   1 twtang  staff   314K Oct 29 06:22 21.zip
-rw-r--r--@  1 twtang  staff   615K Aug 25 18:10 17.zip
-rw-r--r--@  1 twtang  staff   889K Jun 18 19:44 19.zip
-rw-r--r--@  1 twtang  staff   988K Jun 18 19:29 20.zip
-rw-r--r--@  1 twtang  staff   577K Apr 24  2021 18.zip
-rw-r--r--@  1 twtang  staff   569K Jan 29  2021 15.zip
-rw-r--r--@  1 twtang  staff   570K Oct 11  2018 16.zip


In [11]:
sorted(list(path_raw.iterdir()))

[Path('raw_data/.DS_Store'),
 Path('raw_data/15'),
 Path('raw_data/15.zip'),
 Path('raw_data/16'),
 Path('raw_data/16.zip'),
 Path('raw_data/17'),
 Path('raw_data/17.zip'),
 Path('raw_data/18'),
 Path('raw_data/18.zip'),
 Path('raw_data/19'),
 Path('raw_data/19.zip'),
 Path('raw_data/20'),
 Path('raw_data/20.zip'),
 Path('raw_data/21'),
 Path('raw_data/21.zip'),
 Path('raw_data/data.zip')]

In [12]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [str(s).zfill(2) for s in range(15, 22)]

In [13]:
dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [14]:
df = pd.concat(dfs)
df = df.dropna()
df = df.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [15]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df, stats)

In [16]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df, stats)

In [17]:
df = df.merge(df_home, 'left', ['HomeTeam', 'Date']).merge(df_away, 'left', ['AwayTeam', 'Date'])

In [18]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df = df.merge(df_for.rename(columns=cols_home), 'left', ['HomeTeam', 'Date']).merge(df_for.rename(columns=cols_away), 'left', ['AwayTeam', 'Date'])

In [19]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df = df.merge(df_against.rename(columns=cols_home), 'left', ['HomeTeam', 'Date']).merge(df_against.rename(columns=cols_away), 'left', ['AwayTeam', 'Date'])

In [20]:
df = df.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [21]:
df.to_csv(path_data/'data.csv', index=False)

In [22]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HS', 'AS',
       'HST', 'AST', 'HC', 'AC', 'Season', 'FTHGLast5Avg', 'HSLast5Avg',
       'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
       'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
       'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
       'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
       'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
       'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
       'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
       'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg'],
      dtype='object')

In [23]:
df[['HC', 'AC', 'FTHGLast5Avg', 'HSLast5Avg',
    'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
    'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
    'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
    'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
    'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
    'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
    'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
    'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg']].corr()['HC']

HC                        1.000000
AC                       -0.218532
FTHGLast5Avg              0.119302
HSLast5Avg                0.140302
HSTLast5Avg               0.138337
HCLast5Avg                0.145801
FTAGLast5Avg             -0.079451
ASLast5Avg               -0.091699
ASTLast5Avg              -0.084818
ACLast5Avg               -0.068714
HomeFTGForLast5Avg        0.120418
HomeSForLast5Avg          0.144837
HomeSTForLast5Avg         0.139234
HomeCForLast5Avg          0.148493
AwayFTGForLast5Avg       -0.088310
AwaySForLast5Avg         -0.088023
AwaySTForLast5Avg        -0.091943
AwayCForLast5Avg         -0.069260
HomeFTGAgainstLast5Avg   -0.077434
HomeSAgainstLast5Avg     -0.091247
HomeSTAgainstLast5Avg    -0.080784
HomeCAgainstLast5Avg     -0.074606
AwayFTGAgainstLast5Avg    0.098248
AwaySAgainstLast5Avg      0.142409
AwaySTAgainstLast5Avg     0.124777
AwayCAgainstLast5Avg      0.139540
Name: HC, dtype: float64

In [24]:
df[['HC', 'AC', 'FTHGLast5Avg', 'HSLast5Avg',
    'HSTLast5Avg', 'HCLast5Avg', 'FTAGLast5Avg', 'ASLast5Avg',
    'ASTLast5Avg', 'ACLast5Avg', 'HomeFTGForLast5Avg', 'HomeSForLast5Avg',
    'HomeSTForLast5Avg', 'HomeCForLast5Avg', 'AwayFTGForLast5Avg',
    'AwaySForLast5Avg', 'AwaySTForLast5Avg', 'AwayCForLast5Avg',
    'HomeFTGAgainstLast5Avg', 'HomeSAgainstLast5Avg',
    'HomeSTAgainstLast5Avg', 'HomeCAgainstLast5Avg',
    'AwayFTGAgainstLast5Avg', 'AwaySAgainstLast5Avg',
    'AwaySTAgainstLast5Avg', 'AwayCAgainstLast5Avg']].corr()['AC']

HC                       -0.218532
AC                        1.000000
FTHGLast5Avg             -0.078128
HSLast5Avg               -0.081991
HSTLast5Avg              -0.089296
HCLast5Avg               -0.062057
FTAGLast5Avg              0.106553
ASLast5Avg                0.125511
ASTLast5Avg               0.125138
ACLast5Avg                0.132648
HomeFTGForLast5Avg       -0.075198
HomeSForLast5Avg         -0.082847
HomeSTForLast5Avg        -0.088683
HomeCForLast5Avg         -0.058380
AwayFTGForLast5Avg        0.107749
AwaySForLast5Avg          0.129369
AwaySTForLast5Avg         0.129157
AwayCForLast5Avg          0.141153
HomeFTGAgainstLast5Avg    0.097382
HomeSAgainstLast5Avg      0.135543
HomeSTAgainstLast5Avg     0.119613
HomeCAgainstLast5Avg      0.139912
AwayFTGAgainstLast5Avg   -0.073007
AwaySAgainstLast5Avg     -0.083952
AwaySTAgainstLast5Avg    -0.078374
AwayCAgainstLast5Avg     -0.060593
Name: AC, dtype: float64

## END